In [1]:
%load_ext autotime

In [ ]:
import json
from wb_nlp.interfaces import mongodb
from wb_nlp.types import metadata as meta_type
from wb_nlp.types import metadata_enums
import importlib

importlib.reload(meta_type)
importlib.reload(metadata_enums)

# World Bank Metadata

Load the metadata from the API scraper notebook: `wb_nlp/notebooks/scrapers/wb_api_scraper_latest.ipynb`

In [ ]:
# fname = f"wb_metadata-{datetime.now()}.csv"
fname = "wb_metadata-2021-03-27 19:17:44.764136.csv"
wb_metadata = pd.read_csv(dir_manager.get_data_dir("corpus", "WB", fname))

collection = mongodb.get_collection("test_nlp", "docs_metadata")
errors_list = []
no_url = []
metadata_list = []
for i, row in wb_metadata.iterrows():
    if i % 10000 == 0:
        print(i)

    row["_id"] = row["id"]

    if row["url_txt"] == "not to be displayed--":
        print(row["id"])
        no_url.append(row)
        continue

    try:
        meta = json.loads(meta_type.make_metadata_model_from_nlp_schema(row.fillna("")).json())
        meta["_id"] = meta["id"]
        metadata_list.append(meta)
    except:
        errors_list.append(row)

collection.insert_many(metadata_list)